# Advanced guide for data modeling based cogshop runs

**This tutorial covers more advanced techniques than quickstart and will not go into details covered there, such as setting up the Powerops Client**


## Preparation


NOTE: The "recommended" way of running CogShop is as follows:
1. Static model data and mappings for time-dependent data like time series represented by `ShopModel`
2. Any variations on the base configuration (like different price scenarios) represented by `ShopScenario` instances 
3. Generate a complete `ShopCase` with time series datapoints based on a `ShopScenario` instance plus `startTime` and `endTime` through the shop_trigger Cognite function.

This tutorial covers a different set-up: Triggering (Cog)Shop for a pre-generated, complete Shop "case" (as a yaml file or separate files).


In [1]:
# Setting up `PowerOpsClient`
# You can control which setting files are loaded through the environmental variable below.
import os
from cognite.powerops._version import __version__
from cognite.powerops import PowerOpsClient


os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

print(__version__)  # Print the version of the package

0.101.2


In [2]:
# Creating some helper functions

from pathlib import Path


def upload_file(local_file_name: str, external_id: str) -> str:
    """Upload a file to CDF and return the external id of the file"""
    file_path = (Path("example_case_files") / local_file_name).resolve()
    file = powerops.cdf.files.upload(
        path=str(file_path),
        external_id=external_id,
        name=local_file_name,
        data_set_id=powerops.datasets.write_dataset_id,
        mime_type="application/yaml",
        # Overwrite the file at the given external is if it already exists
        # This will also overwrite potentially existing metadata
        overwrite=True,
    )
    return file.external_id

## Example 1: Use multiple files for shop files
Very similar to the example in quick start, but using multiple files for `ShopCase`.

1. Upload the files that will be loaded into shop, keeping their external ids
2. Set up a `ShopCase`, using the SDK
3. Write the `ShopCase` instance to CDF, and verify that it was created using the SDK
4. Trigger a shop execution of that `ShopCase`
5. List the `ShopResult` referencing the `ShopCase` 


### Step 1: Upload a files 

In [26]:

# NB! The case in this example is minimal and just for demo purposes.
example_case_fbu_1 = upload_file(local_file_name="b_example_fornebu_without_commands.yaml", external_id="example_case_fornebu")
example_case_fbu_2 = upload_file(local_file_name="b_example_commands.yaml", external_id="example_commands")

print(f" File reference 1: '{example_case_fbu_1}', File reference 2: '{example_case_fbu_2}'")

 File reference 1: 'example_case_fornebu', File reference 2: 'example_commands'


### Step 2: Preparing a ShopCase

The main difference here is that `shop_file_list` will contain more than one tuple.

Unlike the quickstart, in this example we will let `prepare_shop_case` generate all the external IDs.

We will also let the file reference double as the name of the file. 


In [29]:
import datetime


#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]


shop_case_1 = powerops.cogshop.prepare_shop_case(
    shop_file_list=shop_file_list,
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    model_name="my_model",
    scenario_name="my_scenario",
)
shop_case_1

,value
space,power_ops_instances
external_id,shopcase:4475ca6369ff48cc91f3405f28811d72
data_record,{'existing_version': None}
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00
end_time,2023-01-29 23:00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


In [39]:
# We can also look at just the shop files
for file in shop_case_1.shop_files:
    print(f"File name: '{file.name}', File reference: '{file.file_reference}' \t[,data modeling external id: '{file.external_id}']")

File name: 'example_case_fornebu', File reference: 'example_case_fornebu' 	[,data modeling external id: 'shopfile:d03c6bc81de34ef0889aa5504407a96b']
File name: 'example_commands', File reference: 'example_commands' 	[,data modeling external id: 'shopfile:7ab174df4ba54c1d8e29e688d58b95c1']


### Step 3: Upload the ShopCase

Note: The upload is an `upsert` -- so if there are overlaps, the existing properties will merge  


In [ ]:
powerops.cogshop.write_resources(shop_case_1)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='power_ops_instances', external_id='shopcase:4475ca6369ff48cc91f3405f28811d72', version=1) at 0x7fb1c7d98a90>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:d03c6bc81de34ef0889aa5504407a96b', version=1) at 0x7fb1c67356d0>, <NodeApplyResult(space='power_ops_instances', external_id='shopfile:7ab174df4ba54c1d8e29e688d58b95c1', version=1) at 0x7fb1c6c0f210>, <NodeApplyResult(space='power_ops_instances', external_id='shopscenario:cdb76b8aaec044038b46206943069dc8', version=1) at 0x7fb1c6519c90>, <NodeApplyResult(space='power_ops_instances', external_id='shopmodel:af030fad26834be293d460fa2f06e196', version=1) at 0x7fb1c6c37a50>], edges=[<EdgeApplyResult(space='power_ops_instances', external_id='shopcase:4475ca6369ff48cc91f3405f28811d72:shopfile:d03c6bc81de34ef0889aa5504407a96b', version=1) at 0x7fb1c6731c10>, <EdgeApplyResult(space='power_ops_instances', external_id='shopcase:4475ca6369ff48cc91f3405f28811d72:shopfile:7ab

In [41]:
from cognite.powerops.client._generated.v1.data_classes import ShopCase

# NB! This step is not necessary, if the upsert was successful we know the case is in CDF
retrieved_shop_case: ShopCase = powerops.cogshop.retrieve_shop_case(
    shop_case_1.external_id
)
retrieved_shop_case

,value
space,power_ops_instances
external_id,shopcase:4475ca6369ff48cc91f3405f28811d72
data_record,"{'version': 1, 'last_updated_time': 2024-11-08..."
node_type,None
scenario,shopscenario:cdb76b8aaec044038b46206943069dc8
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[shopfile:d03c6bc81de34ef0889aa5504407a96b, sh..."


Alternately, use Graph QL to retrieve the Shop Case instance. 
Under the hood, this uses a different method to look for the case so the returned data has a slightly different format

In [42]:
graph_ql_shop_case = powerops.cogshop.retrieve_shop_case_graphql(
    shop_case_1.external_id
)
graph_ql_shop_case

,value
space,power_ops_instances
external_id,shopcase:4475ca6369ff48cc91f3405f28811d72
data_record,"{'version': 0, 'last_updated_time': 2024-11-08..."
node_type,None
scenario,"{'space': 'power_ops_instances', 'external_id'..."
start_time,2023-01-19 23:00:00+00:00
end_time,2023-01-29 23:00:00+00:00
shop_files,"[{'space': 'power_ops_instances', 'external_id..."


### Step 4: Trigger the case

We will here directly use the new `trigger_shop_case` method on the `powerops.cogshop` client

In [43]:
powerops.cogshop.trigger_shop_case(shop_case_1.external_id)

### Step 5: View the ShopResult generated 

Because it is the case that a single case can be triggered multiple times, lists will be returned 


#### 1. Using list

Note that it may take a while to run (Cog)Shop. 
If the run is not completed then no results are returned

In [52]:
from cognite.powerops.client._generated.v1.data_classes import ShopResultList

result_list_1: ShopResultList = powerops.cogshop.list_shop_results_for_case(
    case_external_id=shop_case_1.external_id
)
result_list_1

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,power_ops_instances,shop_result__0cde252c-83c5-4ee7-90df-79453986cf77,shopcase:4475ca6369ff48cc91f3405f28811d72,"{'total': -19927822739.279236, 'load_value': 0...",POWEROPS_SHOP_pre-run-0cde252c-83c5-4ee7-90df-...,POWEROPS_SHOP_post-run-0cde252c-83c5-4ee7-90df...,POWEROPS_SHOP_shop-0cde252c-83c5-4ee7-90df-794...,POWEROPS_SHOP_cplex-0cde252c-83c5-4ee7-90df-79...,[shop_penalty_report_my_scenario_2023-01-19 23...,None,None,"{'version': 1, 'last_updated_time': 2024-11-08..."


#### 2. Graph QL list 

In [53]:
from cognite.powerops.client._generated.v1.data_classes import ShopResult

result_list_ql_1: list[ShopResult] = powerops.cogshop.list_shop_result_graphql(
    case_external_id=shop_case_1.external_id, limit=1
)
result_list_ql_1

[ShopResult(space='power_ops_instances', external_id='shop_result__0cde252c-83c5-4ee7-90df-79453986cf77', data_record=DataRecord(version=0, last_updated_time=datetime.datetime(2024, 11, 8, 14, 17, 36, 556000, tzinfo=TzInfo(UTC)), created_time=datetime.datetime(2024, 11, 8, 14, 17, 36, 556000, tzinfo=TzInfo(UTC)), deleted_time=None), node_type=None, objective_value=None, pre_run='POWEROPS_SHOP_pre-run-0cde252c-83c5-4ee7-90df-79453986cf77_aac0a96f', post_run='POWEROPS_SHOP_post-run-0cde252c-83c5-4ee7-90df-79453986cf77', messages='POWEROPS_SHOP_shop-0cde252c-83c5-4ee7-90df-79453986cf77', cplex_logs='POWEROPS_SHOP_cplex-0cde252c-83c5-4ee7-90df-79453986cf77')]

#### 3. Retrieve using the external ID  

In [55]:
shop_result_1:ShopResult = powerops.cogshop.retrieve_shop_result(result_list_ql_1[0].external_id)
shop_result_1

,value
space,power_ops_instances
external_id,shop_result__0cde252c-83c5-4ee7-90df-79453986cf77
data_record,"{'version': 1, 'last_updated_time': 2024-11-08..."
node_type,None
case,shopcase:4475ca6369ff48cc91f3405f28811d72
objective_value,"{'total': -19927822739.279236, 'load_value': 0..."
pre_run,POWEROPS_SHOP_pre-run-0cde252c-83c5-4ee7-90df-...
post_run,POWEROPS_SHOP_post-run-0cde252c-83c5-4ee7-90df...
messages,POWEROPS_SHOP_shop-0cde252c-83c5-4ee7-90df-794...
cplex_logs,POWEROPS_SHOP_cplex-0cde252c-83c5-4ee7-90df-79...


## Example 2: Reusing a ShopScenario used to set shop version

As mentioned in the intro Quickstart, the recommend way to run CogShop is by having a static model and use scenarios to vary configurations. 

However, if we know that we want to run several `ShopCases` using the same configurations, we can reuse a `ShopScenario`/`ShopModel` instead of creating several identical instances.


1. Create a `ShopScenario` and `ShopModel` that set the desired `shop_version`. Here we will create two for demo purposes
2. Write the `ShopScenario`(s) to CDF, and verify that we can retrieve them 
3. Set up a `ShopCase`, preparing shop file list with names and references as explained in Example 1 and Quickstart.
4. Write `ShopCase` instance to CDF, trigger a shop execution of and list the `ShopResult` as in previous examples 

As in the previous example, we will be letting the SDK generate all external id. Again they can be 

### Step 1: Create reusable ShopScenarios with ShopScenarios

`ShopScenarios` and `ShopScenarios` created this way, will have several unused fields that are None. 
Again, this is due to the data models being intended to do more than just trigger a full case


In [4]:
# Not necessary, but we can list available files Shop Version
powerops.cogshop.list_shop_versions()

['SHOP-15.4.1.1-pyshop-python3.11-linux.zip',
 'SHOP-16.0.3-pyshop-python3.11-linux.zip',
 'SHOP-16.0.2-pyshop-python3.11-linux.zip',
 'SHOP-15.6.1.0-pyshop-python3.11-linux.zip']

In [12]:
my_shop_scenario_16_0_3 = powerops.cogshop.prepare_shop_scenario(
    shop_version="16.0.3", 
    model_name="shop_model_16_0_3",
    scenario_name="shop_scenario_16_0_3",
)
# my_shop_scenario_16_0_3

my_shop_scenario_15_5_1_0 = powerops.cogshop.prepare_shop_scenario(
    shop_version="15.5.1.0", 
    model_name="shop_model_15_5_1_0",
    scenario_name="shop_scenario_15_5_1_0",
)

In [13]:
# Looking at the scenarios
my_shop_scenario_16_0_3, my_shop_scenario_15_5_1_0

(ShopScenarioWrite(space='power_ops_instances', external_id='shopscenario:488023ba39774f90a2e20d99071d390c', data_record=DataRecordWrite(existing_version=None), node_type=None, name='shop_scenario_16_0_3', source=None),
 ShopScenarioWrite(space='power_ops_instances', external_id='shopscenario:a05a58e42fad41158ddd60264fd9fec8', data_record=DataRecordWrite(existing_version=None), node_type=None, name='shop_scenario_15_5_1_0', source=None))

In [14]:
# Looking at the models
my_shop_scenario_16_0_3.model, my_shop_scenario_15_5_1_0.model

(ShopModelWrite(space='power_ops_instances', external_id='shopmodel:0eaa852671ae4059b22de68f0d97f48c', data_record=DataRecordWrite(existing_version=None), node_type=DirectRelationReference(space='power_ops_types', external_id='ShopModel'), name='shop_model_16_0_3', model_version=None, shop_version='16.0.3', penalty_limit=None, model=None),
 ShopModelWrite(space='power_ops_instances', external_id='shopmodel:0ec743e5a3844f8b95d8b27f1dcacaf3', data_record=DataRecordWrite(existing_version=None), node_type=DirectRelationReference(space='power_ops_types', external_id='ShopModel'), name='shop_model_15_5_1_0', model_version=None, shop_version='15.5.1.0', penalty_limit=None, model=None))

### Step 2: Write the scenarios to CDF

In [15]:
powerops.cogshop.write_resources(my_shop_scenario_16_0_3, my_shop_scenario_15_5_1_0)

AttributeError: 'CogShopAPI' object has no attribute 'write_resources'

In [ ]:
example_case_svg = upload_file(file_name="a_example_stavanger_with_commands.yaml")

In [ ]:
#  Files are referenced using 4-tuples with the following structure:
# (file_reference, file_name, is_ascii, labels)
shop_file_list = [
    (example_case_fbu_1, example_case_fbu_1, False, ""),
    # CogShop needs to know which file contains commands in the case of multiple files
    (example_case_fbu_2, example_case_fbu_2, False, "commands"),
]


shop_case_1 = powerops.cogshop.prepare_shop_case(
    shop_file_list=shop_file_list,
    shop_version="15.6.1.0",
    start_time=datetime.datetime(2023, 1, 19, 23),
    end_time=datetime.datetime(2023, 1, 29, 23),
    model_name="my_model",
    scenario_name="my_scenario",
)
shop_case_1

## Cleaning up the instances that were created

Instances can be deleted by using the powerops client and the `external_id`s of everything we have created 

In [59]:
all_external_ids = [
    shop_result_1.external_id,
    shop_case_1.external_id,
    shop_case_1.shop_files[0].external_id,
    shop_case_1.shop_files[1].external_id,
    shop_case_1.scenario.external_id,
    shop_case_1.scenario.model.external_id,
]

all_external_ids

['shop_result__0cde252c-83c5-4ee7-90df-79453986cf77',
 'shopcase:4475ca6369ff48cc91f3405f28811d72',
 'shopfile:d03c6bc81de34ef0889aa5504407a96b',
 'shopfile:7ab174df4ba54c1d8e29e688d58b95c1',
 'shopscenario:cdb76b8aaec044038b46206943069dc8',
 'shopmodel:af030fad26834be293d460fa2f06e196']

In [60]:
powerops.v1.delete(external_id=all_external_ids)

InstancesDeleteResult(nodes=[NodeId(space='power_ops_instances', external_id='shop_result__0cde252c-83c5-4ee7-90df-79453986cf77'), NodeId(space='power_ops_instances', external_id='shopcase:4475ca6369ff48cc91f3405f28811d72'), NodeId(space='power_ops_instances', external_id='shopfile:d03c6bc81de34ef0889aa5504407a96b'), NodeId(space='power_ops_instances', external_id='shopfile:7ab174df4ba54c1d8e29e688d58b95c1'), NodeId(space='power_ops_instances', external_id='shopscenario:cdb76b8aaec044038b46206943069dc8'), NodeId(space='power_ops_instances', external_id='shopmodel:af030fad26834be293d460fa2f06e196')], edges=[])